# MusicXML Exploration with Music21
The following code attempts to understand and make notes about how .mxl and .xml files work with Music21.

### Imports

In [10]:
# imports and constants
import music21
from pprint import pprint
from difficulty import *
import warnings
import os
import matplotlib.pyplot as plt
import numpy as np

### Parsing

In [4]:
# score = music21.converter.parse("../music/Beethoven_Symphony_9__Op._125.mxl")

# This used to break, reporting the error:
# MusicXMLImportException: In part (Flauto piccolo), measure (959): found unknown MusicXML type: None
# Then I opened the file in MuseScore, had it ignore the issue, and exported it back out to .mxl, and overwrote the original
# Now, it works but takes FOREVER (generally more than 4 min)
# Additionally, it seems like 
# score = music21.converter.parse("../music/xml/Beethoven_Symphony_9__Op._125.mxl")


# score = music21.converter.parse("../music/xml/Beethoven_Symphony_No._5_Op.67_Mvt._1.mscz.mxl")
# score = music21.converter.parse("../music/xml/Holst__The_Planets__Op._32.mxl")
score = music21.converter.parse("../music/xml/musicalion/mozart_concerto_bassoon_mvt_1.xml")

### Validating XML Files

In [ ]:
# Validate all of scores in the musicalion directory
# assign directory
count_good = 0
count_bad = 0
directory = '../music/xml/musicalion/'
# iterate through all the midi files in the directory
for filename in os.listdir(directory):
    if filename == ".DS_Store":
        continue
    try:
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            score = music21.converter.parse(f)
            count_good += 1
    except:
        print("Error parsing file: " + filename)
        count_bad += 1
    print("-----------------------------------------------------")
print("Good: " + str(count_good))
print("Bad: " + str(count_bad))

### Check for Bassoon Parts (Naming and Part Finding)

In [15]:
def is_bassoon_name(candidate):
    candidate_parts = candidate.split(" ")
    bsn_names = ["bassoon", "fagotti", "basson", "fagott", "fagotto", "fagot", "fagote", "dulcian"]
    for name in bsn_names:
        for part in candidate_parts:
            if part == name:
                return True
    return False

In [13]:
# find bassoon part in music21 score
import warnings


bsn_part = None
for part in score.parts:
    # print(part)
    # print(part.getInstrument().instrumentName)
    # print(part.getInstrument())
    # if is_bassoon_name(part.partName.lower()):
    instrument_name = part.getInstrument().instrumentName
    if instrument_name == "Bassoon":
        print(part.partName)
        bsn_part = part
        break
    if instrument_name == "Midi_71":
        part_name = part.getInstrument()
        warnings.warn("\nUsed Midi fallback detection (Midi_71)\nActual Part Name: {}".format(part_name))
        bsn_part = part
        break

if bsn_part == None:
    raise Exception("Bassoon part not found")

/var/folders/67/z2c_5yw12tbcjdhcbfkqwg0c0000gn/T/ipykernel_48477/2420026932.py:18: UserWarning: 
Used Midi fallback detection (Midi_71)
Actual Part Name: P3: Midi_71
  warnings.warn("\nUsed Midi fallback detection (Midi_71)\nActual Part Name: {}".format(part_name))


### Visualize the Data

In [ ]:
score_list = []

# for part in score.parts:
#     instrument = part.getInstrument().instrumentName

for note in bsn_part.flat.notes:

    if note.isChord:
        start = note.offset
        duration = note.quarterLength

        for chord_note in note.pitches:
            pitch = chord_note.ps
            volume = note.volume.realized
            score_list.append([start, duration, pitch, volume])

    else:
        start = note.offset
        duration = note.quarterLength
        pitch = note.pitch.ps
        volume = note.volume.realized
        score_list.append([start, duration, pitch, volume])


# score_list = sorted(score, key=lambda x: (x[0], x[2]))
print(score_list)


In [12]:
data = np.cumsum(np.random.rand(1000)-0.5)

print(data)
data = data - np.mean(data)

[ 0.35239305 -0.03992469 -0.39476467 -0.23130542 -0.57333249 -0.4466214
 -0.60620922 -0.47127639 -0.64333561 -0.52593583 -0.75928201 -0.36974339
 -0.69307937 -0.5834886  -0.82375013 -0.95301542 -0.86187697 -1.0459116
 -1.08751131 -0.8966246  -0.93936661 -0.62404216 -0.31182839 -0.19692114
 -0.5593794  -0.80948784 -0.73289584 -0.77543371 -0.27705807 -0.04737797
  0.16498376  0.56352508  0.31537752  0.61798336  0.98942906  0.88917301
  1.10818183  0.82157908  0.51024363  0.91570843  1.17727953  0.87482765
  0.67196827  0.67288206  0.40134512  0.5446126   0.24076845 -0.23898238
  0.17790341  0.04933409 -0.23950951  0.06102256 -0.05718528 -0.43393025
 -0.51515719 -0.8952183  -1.34757463 -1.48709957 -1.0425729  -1.18871533
 -1.51778328 -1.20383294 -0.93252907 -1.07374065 -0.67419719 -0.35521144
 -0.2760537  -0.60824967 -0.2209233  -0.11983374  0.32808761  0.74291275
  1.16640935  1.01398224  1.14344253  0.92448321  0.97610604  0.85056181
  1.23046404  1.23258871  1.22280659  1.47736146  1.4

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(411) # nrows, ncols, plot_number, top sparkline
ax1.plot(score_list, 'b-')
# ax1.axhline(c='grey', alpha=0.5)

ax2 = fig.add_subplot(412, sharex=ax1) 
ax2.plot(score_list, 'g-')
# ax2.axhline(c='grey', alpha=0.5)

ax3 = fig.add_subplot(413, sharex=ax1)
ax3.plot(score_list, 'y-')
# ax3.axhline(c='grey', alpha=0.5)

ax4 = fig.add_subplot(414, sharex=ax1) # bottom sparkline
ax4.plot(score_list, 'r-')
# ax4.axhline(c='grey', alpha=0.5)


for axes in [ax1, ax2, ax3, ax4]: # remove all borders
    plt.setp(axes.get_xticklabels(), visible=False)
    plt.setp(axes.get_yticklabels(), visible=False)
    plt.setp(axes.get_xticklines(), visible=False)
    plt.setp(axes.get_yticklines(), visible=False)
    plt.setp(axes.spines.values(), visible=False)


# bottom sparkline
plt.setp(ax4.get_xticklabels(), visible=True)
plt.setp(ax4.get_xticklines(), visible=True)
ax4.xaxis.tick_bottom() # but onlyt the lower x ticks not x ticks at the top

plt.tight_layout()
plt.show()

In [57]:
bsn_part.show('musicXML')
# bsn_part.show()

In [ ]:
excerpt = bsn_part.measures(0, 30)
# excerpt.show()
for note in excerpt.flat:
    print(note.next)

In [45]:
# adds a note to a dict if not in it, increments if is in it
def note_record(note, notes):
    if note.nameWithOctave in notes:
        notes[note.nameWithOctave] += 1
        # notes[note.pitch] += 1
    else:
        notes[note.nameWithOctave] = 1
        # notes[note.pitch] = 1

# def note_record(note):
#     if note.pitch in notes_dict:
#         notes_dict[note.pitch] += 1
#     else:
#         notes_dict[note.pitch] = 1
        # if note.nameWithOctave == 'F#2':
        #     pprint(note.__dict__)

def inc_accidental(note, num):
    # check if it's in the key
    if note.pitch.accidental is not None:
        return num + 1
    return num


In [46]:
# Go through score and record the number of occurrences of each note
notes_dict_bassoon_1 = {}
note_pairs_dict_b1 = {}
notes_dict_bassoon_2 = {}
note_pairs_dict_b2 = {}
num_accidentals = 0
# fsharp = music21.note.Note('F#2')

for el in bsn_part.recurse().notes:
    if type(el)== music21.chord.Chord:
        sortedNotes = el.sortAscending()
        note_record(sortedNotes[0], notes_dict_bassoon_2)
        note_record(sortedNotes[1], notes_dict_bassoon_1)
        # for note in el.notes:
        #     note_record(note)
        #     num_accidentals = inc_accidental(note, num_accidentals)
                
    else:
        note_record(el, notes_dict_bassoon_2)
        note_record(el, notes_dict_bassoon_1)
        # note_record(el)
        # num_accidentals = inc_accidental(el, num_accidentals)

print("Number of accidentals:", num_accidentals) # I think this is wrong
print("-------------- Bassoon 1 --------------")
pprint(notes_dict_bassoon_1)
print("-------------- Bassoon 2 --------------")
pprint(notes_dict_bassoon_2)

Number of accidentals: 0
-------------- Bassoon 1 --------------
{'A-2': 17,
 'A-3': 22,
 'A-4': 3,
 'A2': 5,
 'A3': 32,
 'A4': 21,
 'B-2': 26,
 'B-3': 58,
 'B2': 1,
 'B3': 7,
 'C#4': 4,
 'C2': 4,
 'C3': 29,
 'C4': 91,
 'D-4': 71,
 'D3': 2,
 'D4': 17,
 'E-3': 3,
 'E-4': 27,
 'E3': 3,
 'E4': 12,
 'F#3': 2,
 'F#4': 5,
 'F2': 42,
 'F3': 39,
 'F4': 45,
 'G#3': 2,
 'G-3': 3,
 'G-4': 5,
 'G2': 6,
 'G3': 24,
 'G4': 50}
-------------- Bassoon 2 --------------
{'A-2': 18,
 'A-3': 42,
 'A2': 5,
 'A3': 48,
 'A4': 5,
 'B-2': 29,
 'B-3': 36,
 'B2': 1,
 'B3': 5,
 'C#3': 3,
 'C#4': 2,
 'C2': 18,
 'C3': 17,
 'C4': 76,
 'D-3': 17,
 'D-4': 33,
 'D3': 6,
 'D4': 10,
 'E-2': 1,
 'E-3': 12,
 'E-4': 24,
 'E3': 12,
 'E4': 26,
 'F#3': 7,
 'F#4': 4,
 'F-3': 1,
 'F2': 57,
 'F3': 40,
 'F4': 38,
 'G#3': 2,
 'G-3': 10,
 'G-4': 2,
 'G2': 6,
 'G3': 56,
 'G4': 9}
